### Building

In [5]:
import torch
from datasets import load_dataset_builder, load_dataset, get_dataset_split_names
from Model import BertModel
import torch.nn as nn

ModuleNotFoundError: No module named 'Model'

In [28]:
from transformers import BertTokenizerFast, data


In [5]:
hidden_size = 6
vocab_size = 30522
d_ff = 3
n_layers = 2
n_heads = 2
max_len = 512
pad_idx = 0

In [6]:
model = BertModel(hidden_size, vocab_size, d_ff, n_layers, n_heads, max_len, type_vocab_size=2, pad_idx=pad_idx, 
                layer_norm_eps=1e-12, dropout_p=0.1)

In [20]:
dataset_builder = load_dataset_builder('glue', 'rte')
print(dataset_builder.cache_dir)
print(dataset_builder.info.features)
print(dataset_builder.info.splits)

C:\Users\dilab_desktop\.cache\huggingface\datasets\glue\rte\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad
{'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['entailment', 'not_entailment'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}
{'test': SplitInfo(name='test', num_bytes=975936, num_examples=3000, dataset_name='glue'), 'train': SplitInfo(name='train', num_bytes=848888, num_examples=2490, dataset_name='glue'), 'validation': SplitInfo(name='validation', num_bytes=90911, num_examples=277, dataset_name='glue')}


In [4]:
rte_val = load_dataset('glue', 'rte', split='validation')
rte_val

Reusing dataset glue (C:\Users\dilab_desktop\.cache\huggingface\datasets\glue\rte\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 277
})

In [27]:
print(rte_val[0])

{'sentence1': 'Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation.', 'sentence2': 'Christopher Reeve had an accident.', 'label': 1, 'idx': 0}


In [29]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading: 100%|██████████| 226k/226k [00:00<00:00, 301kB/s]  
Downloading: 100%|██████████| 455k/455k [00:00<00:00, 494kB/s]  
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 14.3kB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 286kB/s]


In [54]:
ex = rte_val[0]
ex['sentence2']

'Christopher Reeve had an accident.'

In [52]:
inputs = tokenizer(ex['sentence1'], ex['sentence2'], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
inputs["token_type_ids"] = inputs["token_type_ids"]+(1-inputs["attention_mask"])*2

In [76]:
class BertClassification(nn.Module):
    def __init__(self, model, hidden_size, num_labels, dropout_p=0.1):
        super().__init__()
        self.model = model
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.dropout = nn.Dropout(dropout_p)
        
        self._init_weight()

    def _init_weight(self):
        nn.init.kaiming_uniform_(self.classifier.weight, mode='fan_out', nonlinearity='relu')
        self.classifier.bias.data.zero_()

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_out = self.model(input_ids, attention_mask, token_type_ids)
        return self.classifier(self.dropout(pooled_out))
        

In [77]:
classification = BertClassification(model, hidden_size, num_labels=2)

In [78]:
class_out = classification(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'])

In [79]:
print(class_out)
print(class_out.shape)

tensor([[-3.8730, -2.0079]], grad_fn=<AddmmBackward>)
torch.Size([1, 2])


In [80]:
criterion = nn.CrossEntropyLoss()

In [81]:
label = torch.tensor(ex['label'])

In [82]:
num_class=2
loss = criterion(class_out.view(-1, num_class), label.view(-1))
print(loss)

tensor(0.1440, grad_fn=<NllLossBackward>)


In [83]:
loss.backward()

In [84]:
optimizer = torch.optim.Adam(classification.parameters(), lr=5e-5, betas=(0.9, 0.999), weight_decay=0.01)

In [85]:
optimizer.zero_grad()
optimizer.step()

### Preprocessing

In [1]:
import torch
from datasets import load_dataset

In [3]:
rte_val = load_dataset('glue', 'rte', split='validation')

Reusing dataset glue (C:\Users\dilab_desktop\.cache\huggingface\datasets\glue\rte\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [69]:
def generator(data, name):
    for one_fold in data:
            yield one_fold[f'{name}']

In [12]:
rte_val[0]

{'sentence1': 'Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation.',
 'sentence2': 'Christopher Reeve had an accident.',
 'label': 1,
 'idx': 0}

In [70]:
rte_s1 = list(map(str, generator(rte_val, 'sentence1')))
rte_s2 = list(map(str, generator(rte_val, 'sentence2')))
rte_label = list(map(int, generator(rte_val, 'label')))

In [71]:
rte_s1[:2]

['Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation.',
 'Yet, we now are discovering that antibiotics are losing their effectiveness against illness. Disease-causing bacteria are mutating faster than we can come up with new antibiotics to fight the new variations.']

In [72]:
rte_s2[:2]

['Christopher Reeve had an accident.',
 'Bacteria is winning the war against antibiotics.']

In [73]:
rte_val[:2]

{'sentence1': ['Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation.',
  'Yet, we now are discovering that antibiotics are losing their effectiveness against illness. Disease-causing bacteria are mutating faster than we can come up with new antibiotics to fight the new variations.'],
 'sentence2': ['Christopher Reeve had an accident.',
  'Bacteria is winning the war against antibiotics.'],
 'label': [1, 0],
 'idx': [0, 1]}

In [77]:
len(rte_val)

277

In [44]:
rte_label[:5]

[1, 0, 1, 1, 0]

In [65]:
from torch.utils.data import Dataset

In [87]:
class FineTuningDataset(Dataset):
    def __init__(self, data, max_len):
        self.data = data
        self.sent1 = list(map(str, generator(data, 'sentence1')))
        self.sent2 = list(map(str, generator(data, 'sentence2')))
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs = tokenizer(self.sent1[idx], self.sent2[idx], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        inputs["token_type_ids"] = inputs["token_type_ids"]+(1-inputs["attention_mask"])*2

        return inputs["input_ids"], inputs["attention_mask"], inputs["token_type_ids"]

    def generator(self, data, name):
        for one_fold in data:
            yield one_fold[f'{name}']

In [88]:
from torch.utils.data import DataLoader

In [90]:
dataset = FineTuningDataset(rte_val, len(rte_val))
data_loader = DataLoader(dataset, batch_size=2, shuffle=False)

In [91]:
for i1, i2, i3 in data_loader:
    print(i1)
    print(i1.shape)
    print(i2)
    print(i2.shape)
    print(i3)
    print(i3.shape)
    break

tensor([[[  101, 11271, 20726,  ...,     0,     0,     0]],

        [[  101,  2664,  1010,  ...,     0,     0,     0]]])
torch.Size([2, 1, 512])
tensor([[[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]]])
torch.Size([2, 1, 512])
tensor([[[0, 0, 0,  ..., 2, 2, 2]],

        [[0, 0, 0,  ..., 2, 2, 2]]])
torch.Size([2, 1, 512])


### Testing

In [2]:
import torch
from datasets import load_dataset
import torch.nn as nn
from tqdm.auto import tqdm
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import sys
# 상위 폴더 접근
sys.path.append(r'D:\youjin\OneDrive - 고려대학교\Freshman\BERT')
from transformers import BertTokenizerFast 
from Model import BertModel

In [3]:
hidden_size = 6
vocab_size = 30522
d_ff = 3
n_layers = 2
n_heads = 2
max_len = 512
pad_idx = 0

In [4]:
model = BertModel(hidden_size, vocab_size, d_ff, n_layers, n_heads, max_len, type_vocab_size=2, pad_idx=pad_idx, 
                layer_norm_eps=1e-12, dropout_p=0.1)

In [4]:
class BertClassification(nn.Module):
    def __init__(self, model, hidden_size, num_labels, dropout_p=0.1):
        super().__init__()
        self.model = model
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.dropout = nn.Dropout(dropout_p)
        
        self._init_weight()

    def _init_weight(self):
        nn.init.kaiming_uniform_(self.classifier.weight, mode='fan_out', nonlinearity='relu')
        self.classifier.bias.data.zero_()

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_out = self.model(input_ids, attention_mask, token_type_ids)
        return self.classifier(self.dropout(pooled_out))
        

In [5]:
ft_model = BertClassification(model, hidden_size, num_labels=2)

In [24]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

class FineTuningDataset(Dataset):
    def __init__(self, data, max_len):
        self.data = data
        self.sent1 = list(map(str, self.generator(data, 'sentence1')))
        self.sent2 = list(map(str, self.generator(data, 'sentence2')))
        self.label = list(map(int, self.generator(data, 'label')))
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs = tokenizer(self.sent1[idx], self.sent2[idx], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        inputs["token_type_ids"] = inputs["token_type_ids"]+(1-inputs["attention_mask"])*2
        label = torch.tensor(self.label[idx])

        return inputs["input_ids"], inputs["attention_mask"], inputs["token_type_ids"], label

    def generator(self, data, name):
        for one_fold in data:
            yield one_fold[f'{name}']

In [25]:
rte_val = load_dataset('glue', 'rte', split='validation')

Reusing dataset glue (C:\Users\dilab_desktop\.cache\huggingface\datasets\glue\rte\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [26]:
dataset = FineTuningDataset(rte_val, len(rte_val))
data_loader = DataLoader(dataset, batch_size=2, shuffle=False)

In [27]:
classification = BertClassification(model, hidden_size, num_labels=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classification.parameters(), lr=5e-5, betas=(0.9, 0.999), weight_decay=0.01)

In [33]:
num_class = 2
cnt = 0
for input_ids, attention_mask, token_type_ids, label in tqdm(data_loader):
    input_ids = input_ids.squeeze(1)
    attention_mask = attention_mask.squeeze(1)
    token_type_ids = token_type_ids.squeeze(1)

    class_out = classification(input_ids, attention_mask, token_type_ids)
    loss = criterion(class_out.view(-1, num_class), label.view(-1))
    loss.backward()
    optimizer.zero_grad()
    optimizer.step()
    cnt += 1
    if cnt % 50 == 1:
        print(f"Loss: {loss}")
print(f"Loss: {loss}")

  3%|▎         | 4/139 [00:00<00:08, 15.95it/s]

Loss: 0.8157970905303955


 39%|███▉      | 54/139 [00:03<00:05, 16.15it/s]

Loss: 0.08400219678878784


 75%|███████▍  | 104/139 [00:06<00:02, 16.30it/s]

Loss: 0.027380656450986862


100%|██████████| 139/139 [00:08<00:00, 15.98it/s]


In [6]:
import torch
from datasets import load_dataset_builder, load_dataset, get_dataset_split_names
import torch.nn as nn

In [7]:
dataset_builder = load_dataset_builder('glue', 'mrpc')
print(dataset_builder.cache_dir)
print(dataset_builder.info.features)
print(dataset_builder.info.splits)

C:\Users\dilab_desktop\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad
{'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}
{'test': SplitInfo(name='test', num_bytes=443498, num_examples=1725, dataset_name='glue'), 'train': SplitInfo(name='train', num_bytes=946146, num_examples=3668, dataset_name='glue'), 'validation': SplitInfo(name='validation', num_bytes=106142, num_examples=408, dataset_name='glue')}


In [16]:
class BertSTSB(nn.Module):
    def __init__(self, model, hidden_size, dropout_p=0.1):
        super().__init__()
        self.model = model
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout_p)
        
        self._init_weight()

    def _init_weight(self):
        nn.init.kaiming_uniform_(self.dense.weight, mode='fan_out', nonlinearity='relu')
        self.dense.bias.data.zero_()
        nn.init.kaiming_uniform_(self.linear.weight, mode='fan_out', nonlinearity='relu')
        self.linear.bias.data.zero_()

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_out = self.model(input_ids, attention_mask, token_type_ids)
        pooled_out = nn.functional.gelu(self.dense(pooled_out))
        return self.linear(self.dropout(pooled_out))

In [28]:
from torch.utils.data import Dataset
import torch

from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# use huggingfase tokenizer
class ClassificationDataset(Dataset):
    def __init__(self, data, tag):
        self.tag = tag
        if isinstance(tag, tuple):
            tag1, tag2 = tag
            self.sent1 = list(map(str, self.generator(data, f'{tag1}')))
            self.sent2 = list(map(str, self.generator(data, f'{tag2}')))
        else:
            self.sent = list(map(str, self.generator(data, f'{tag}')))
        self.label = list(map(int, self.generator(data, 'label')))
    
    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        if isinstance(self.tag, tuple):
            inputs = tokenizer(self.sent1[idx], self.sent2[idx], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        else:
            inputs = tokenizer(self.sent[idx], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        inputs["token_type_ids"] = inputs["token_type_ids"]+(1-inputs["attention_mask"])*2
        label = torch.tensor(self.label[idx])

        return inputs["input_ids"], inputs["attention_mask"], inputs["token_type_ids"], label

    def generator(self, data, name):
        for one_fold in data:
            yield one_fold[f'{name}']

In [7]:
stsb_val = load_dataset('glue', 'stsb', split='validation')

Reusing dataset glue (C:\Users\dilab_desktop\.cache\huggingface\datasets\glue\stsb\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [25]:
dataset = ClassificationDataset(stsb_val, 512, ('sentence1', 'sentence2'))
data_loader = DataLoader(dataset, batch_size=2, shuffle=False)

model = BertModel(hidden_size, vocab_size, d_ff, n_layers, n_heads, max_len, type_vocab_size=2, pad_idx=pad_idx, 
                layer_norm_eps=1e-12, dropout_p=0.1)
stsb_model = BertSTSB(model, hidden_size)
criterion = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(stsb_model.parameters(), lr=5e-5, betas=(0.9, 0.999), weight_decay=0.01)

In [29]:
for epoch in range(2):
    for input, attention_mask, token_type_ids, label in tqdm(data_loader):
        input = input.squeeze(1)
        attention_mask = attention_mask.squeeze(1)
        token_type_ids = token_type_ids.squeeze(1)
        label = label.to(dtype=torch.float)

        optimizer.zero_grad()
        stsb_out = stsb_model(input, attention_mask, token_type_ids)
        loss = criterion(stsb_out.view(-1), label.view(-1)) / input.size(0)
        loss.backward()
        optimizer.step()

  0%|          | 0/750 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [5] at entry 0 and [4] at entry 1

### Check Metric

In [34]:
from datasets import load_metric

In [10]:
f1_metric = load_metric('f1')

In [39]:
pred = torch.tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label = torch.tensor([1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0])

NameError: name 'torch' is not defined

In [12]:
result = f1_metric.compute(predictions=pred, references=label)
result

{'f1': 0.33333333333333337}

In [13]:
print(f1_metric.compute(predictions=pred, references=label, average='macro'))
print(f1_metric.compute(predictions=pred, references=label, average='micro'))
print(f1_metric.compute(predictions=pred, references=label, average='weighted'))
print(f1_metric.compute(predictions=pred, references=label, average=None))

{'f1': 0.4666666666666667}
{'f1': 0.5}
{'f1': 0.43333333333333335}
{'f1': array([0.6       , 0.33333333])}


In [40]:
pred1 = [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
label1 = [1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0]

In [15]:
print(f1_metric.compute(predictions=pred1, references=label1, average='macro'))
print(f1_metric.compute(predictions=pred1, references=label1, average='micro'))
print(f1_metric.compute(predictions=pred1, references=label1, average='weighted'))
print(f1_metric.compute(predictions=pred1, references=label1, average=None))

{'f1': 0.4666666666666667}
{'f1': 0.5}
{'f1': 0.43333333333333335}
{'f1': array([0.6       , 0.33333333])}


In [16]:
acc_metric = load_metric('accuracy')

Downloading: 3.20kB [00:00, 642kB/s]                    


In [20]:
acc_metric.compute(predictions=pred, references=label)

{'accuracy': 0.5}

In [37]:
pearsonr_metric = load_metric('pearsonr')

In [41]:
pearsonr_metric.compute(predictions=pred1, references=label1)

{'pearsonr': 0.2927700218845599}

# Collection of Number of Labels for GLUE tasks

In [4]:
from datasets import load_dataset_builder,get_dataset_split_names

In [26]:
dataset_builder = load_dataset_builder('glue', 'rte')
print(dataset_builder.info.features)
print(get_dataset_split_names('glue', 'rte'))
print("Accuracy")

{'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['entailment', 'not_entailment'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}
['train', 'validation', 'test']
Accuracy


In [27]:
dataset_builder = load_dataset_builder('glue', 'mrpc')
print(dataset_builder.info.features)
print(get_dataset_split_names('glue', 'mrpc'))
print("Accuracy / F1")

{'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}
['test', 'train', 'validation']
Accuracy / F1


In [28]:
dataset_builder = load_dataset_builder('glue', 'qqp')
print(dataset_builder.info.features)
print(get_dataset_split_names('glue', 'qqp'))
print("Accuracy / F1")

{'question1': Value(dtype='string', id=None), 'question2': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['not_duplicate', 'duplicate'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}
['train', 'validation', 'test']
Accuracy / F1


In [13]:
dataset_builder = load_dataset_builder('glue', 'qnli')
print(dataset_builder.info.features)
print(get_dataset_split_names('glue', 'qnli'))

{'question': Value(dtype='string', id=None), 'sentence': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['entailment', 'not_entailment'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}
['test', 'train', 'validation']


In [29]:
dataset_builder = load_dataset_builder('glue', 'sst2')
print(dataset_builder.info.features)
print(get_dataset_split_names('glue', 'sst2'))
print("Accuracy")

{'sentence': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['negative', 'positive'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}
['test', 'train', 'validation']
Accuracy


In [42]:
dataset_builder = load_dataset_builder('glue', 'stsb')
print(dataset_builder.info.features)
print(get_dataset_split_names('glue', 'stsb'))
print("Pearson correlation")

{'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': Value(dtype='float32', id=None), 'idx': Value(dtype='int32', id=None)}
['test', 'train', 'validation']
Pearson correlation


In [31]:
dataset_builder = load_dataset_builder('glue', 'mnli_matched')
print(dataset_builder.info.features)
print(get_dataset_split_names('glue', 'mnli_matched'))
print("Accuracy")

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=3, names=['entailment', 'neutral', 'contradiction'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}
['test', 'validation']
Accuracy


In [30]:
dataset_builder = load_dataset_builder('glue', 'mnli_mismatched')
print(dataset_builder.info.features)
print(get_dataset_split_names('glue', 'mnli_mismatched'))
print("Accuracy")

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=3, names=['entailment', 'neutral', 'contradiction'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}
['test', 'validation']
Accuracy


In [1]:
from datasets import load_dataset

In [3]:
stsb_data = load_dataset('glue', 'stsb')
print(stsb_data['validation']['label'][:5])

Reusing dataset glue (C:\Users\dilab_desktop\.cache\huggingface\datasets\glue\stsb\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 601.65it/s]

[5.0, 4.75, 5.0, 2.4000000953674316, 2.75]


In [5]:
stsb_data = load_dataset('glue', 'sst2')
print(stsb_data['validation'][:5])

Downloading: 100%|██████████| 7.44M/7.44M [00:01<00:00, 6.97MB/s]


Dataset glue downloaded and prepared to C:\Users\dilab_desktop\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 88.47it/s]

{'sentence': ["it 's a charming and often affecting journey . ", 'unflinchingly bleak and desperate ', 'allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . ', "the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . ", "it 's slow -- very , very slow . "], 'label': [1, 0, 1, 1, 0], 'idx': [0, 1, 2, 3, 4]}
